In [1]:
import google.generativeai as genai
from dotenv import load_dotenv
import os
import requests
from io import BytesIO
import glob
import os
from IPython.display import Markdown, display

# Load environment variables from the .env file
load_dotenv()

# Retrieve the API key from the environment variable
GOOGLE_API_KEY = os.getenv('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)
generation_config = genai.GenerationConfig(
    temperature=0.0,
    response_mime_type="application/json"
)

In [2]:
system_instruction_prompt= """
You are a Cognitive Specialist AI. Your task is to refer to the attached papers, then generate factual, score‑driven descriptions for a “Child Profile” JSON object. Do not invent data or hallucinate—only use established research-based language.  

When prompted, you will receive:  
1. One or more uploaded PDF resource papers.  
2. A JSON skeleton with numeric scores and empty description fields (as in the example below).  

Your job is to populate every `"description"` or summary and also modalities of the kids profile field based strictly on the score values and established cognitive‑domain definitions in the reference material. Output **only** the completed JSON object—no extra commentary, no markdown, and no omitted fields.  

Example input schema:
```json
{
  "name": "Kid",
  "gender": "",
  "age": 1,
  "diagnosis": "not identified",
  "cognitiveDomainProfile": {
    "workingMemory": {
      "score": 72,
      "description": ""
    },
    "processingSpeed": {
      "score": 68,
      "description": ""
    },
    "attentionSpan": {
      "visualAttention": {
        "score": 65,
        "description": ""
      },
      "auditoryAttention": {
        "score": 70,
        "description": ""
      }
    }
  },
  "executiveFunction": {
    "planning": 4,
    "impulseControl": 3,
    "description": ""
  },
  "learningPreferences": {
    "modalities": [],
    "description": ""
  },
  "learningChallengesSummary": {
    "attentionAndFocus": "",
    "memoryAndSpeed": "",
    "planningAndImpulsivity": "",
    "strengths": ""
  }
}


 """

In [3]:
profile1 = {
  "name": "Kid",
  "gender": "male",
  "age": 13,
  "diagnosis": "Not formally diagnosed, but exhibiting early symptoms suggestive of attention deficit hyperactivity disorder (ADHD)",
  "cognitiveDomainProfile": {
    "workingMemory": {
      "score": 85,
      "description": ""
    },
    "processingSpeed": {
      "score": 78,
      "description": ""
    },
    "attentionSpan": {
      "visualAttention": {
        "score": 80,
        "description": ""
      },
      "auditoryAttention": {
        "score": 75,
        "description": ""
      }
    }
  },
  "executiveFunction": {
    "planning": 6,
    "impulseControl": 6,
    "description": ""
  },
  "learningPreferences": {
    "modalities": [],
    "description": ""
  },
  "learningChallengesSummary": {
    "attentionAndFocus": "",
    "memoryAndSpeed": "",
    "planningAndImpulsivity": "",
    "strengths": ""
  }
}


profile2= {
  "name": "Kid",
  "gender": "female",
  "age": 13,
  "diagnosis": "Attention Deficit Hyperactivity Disorder (ADHD)",
  "cognitiveDomainProfile": {
    "workingMemory": {
      "score": 85,
      "description": ""
    },
    "processingSpeed": {
      "score": 78,
      "description": ""
    },
    "attentionSpan": {
      "visualAttention": {
        "score": 80,
        "description": ""
      },
      "auditoryAttention": {
        "score": 75,
        "description": ""
      }
    }
  },
  "executiveFunction": {
    "planning": 6,
    "impulseControl": 6,
    "description": ""
  },
  "learningPreferences": {
    "modalities": [],
    "description": ""
  },
  "learningChallengesSummary": {
    "attentionAndFocus": "",
    "memoryAndSpeed": "",
    "planningAndImpulsivity": "",
    "strengths": ""
  }
}

profile3= {
  "name": "Kid",
  "gender": "female",
  "age": 13,
  "diagnosis": "Identified as Gifted and Talented (IQ > 130)",
  "cognitiveDomainProfile": {
    "workingMemory": {
      "score": 135,
      "description": ""
    },
    "processingSpeed": {
      "score": 140,
      "description": ""
    },
    "attentionSpan": {
      "visualAttention": {
        "score": 145,
        "description": ""
      },
      "auditoryAttention": {
        "score": 142,
        "description": ""
      }
    }
  },
  "executiveFunction": {
    "planning": 10,
    "impulseControl": 9,
    "description": ""
  },
  "learningPreferences": {
    "modalities": [],
    "description": ""
  },
  "learningChallengesSummary": {
    "attentionAndFocus": "",
    "memoryAndSpeed": "",
    "planningAndImpulsivity": "",
    "strengths": ""
  }
}


profile4 = {
  "name": "College Student",
  "gender": "female",
  "age": 21,
  "diagnosis": "Identified as Gifted and Talented (IQ > 130)",
  "cognitiveDomainProfile": {
    "workingMemory": {
      "score": 145,
      "description": "Exhibits exceptional working memory, retaining and manipulating multiple complex ideas simultaneously—great for advanced coursework and research."
    },
    "processingSpeed": {
      "score": 150,
      "description": "Processes new information extremely quickly, routinely completing readings and problem sets faster than peers."
    },
    "attentionSpan": {
      "visualAttention": {
        "score": 148,
        "description": "Maintains deep focus on visual materials (diagrams, slides, code) for extended periods without fatigue."
      },
      "auditoryAttention": {
        "score": 149,
        "description": "Absorbs and integrates spoken lectures and discussions rapidly, rarely missing nuanced details even in fast‑paced seminars."
      }
    }
  },
  "executiveFunction": {
    "planning": 10,
    "impulseControl": 10,
    "description": "Demonstrates immaculate organizational skills and self‑regulation, balancing a full course load, research projects, and extracurriculars with ease."
  },
  "learningPreferences": {
    "modalities": ["Visual", "Auditory", "Kinesthetic"],
    "description": "Thrives on a mix of interactive lectures, hands‑on labs, and visual aids—often sketching mind‑maps or models to solidify understanding."
  },
  "learningChallengesSummary": {
    "attentionAndFocus": "Rarely distracted, though finds repetitive tasks unstimulating and may seek novelty.",
    "memoryAndSpeed": "Outstanding recall of concepts and facts; occasionally experiences overload when retrieving very granular details.",
    "planningAndImpulsivity": "Excels at long‑term planning; may procrastinate on assignments perceived as low‑challenge.",
    "strengths": "Creative problem‑solving, rapid insight generation, and advanced pattern recognition across disciplines."
  }
}

In [4]:
pattern = os.path.join("papers_profiles", "*.pdf")
pdf_paths = glob.glob(pattern)

print(pdf_paths)

['papers_profiles\\Assessing executive functions in children.pdf', 'papers_profiles\\Assessments Related to the Physical, Affective and Cognitive Domains of Physical Literacy.pdf', 'papers_profiles\\Auditory and Visual Attention in Normal and ADHD-Inattentive subtype Children.pdf', 'papers_profiles\\Domains of cognition and their assessment.pdf', 'papers_profiles\\Evidence-based Assessment of Cognitive Functioning.pdf', 'papers_profiles\\Mapping Normative Trajectories of Cognitive.pdf', 'papers_profiles\\Measurement of Cognition for the.pdf', 'papers_profiles\\wisc-v-interpretive-report.pdf', 'papers_profiles\\wms-iv-interpretive-report-sample-female.pdf']


In [5]:
model = genai.GenerativeModel(
    model_name='models/gemini-1.5-pro',           
    system_instruction=system_instruction_prompt,  
    generation_config=generation_config,
       
)

# pdf_paths = [
#     "papers_profiles\Assessing executive functions in children.pdf",
#     "papers_profiles\Assessments Related to the Physical, Affective and Cognitive Domains of Physical Literacy.pdf",
#     "papers_profiles\Mapping Normative Trajectories of Cognitive.pdf",
#     "papers_profiles\Domains of cognition and their assessment.pdf",
#     "papers_profiles\Evidence-based Assessment of Cognitive Functioning.pdf",
#     "papers_profiles\Measurement of Cognition for the.pdf",
#     "papers_profiles\Auditory and Visual Attention in Normal and ADHD-Inattentive subtype Children.pdf"
# ]


# build the pattern to match all .pdf files in that folder


# 5) Upload each PDF
pdf_files = [
    genai.upload_file(path=path, mime_type="application/pdf")
    for path in pdf_paths
]  # returns a list of File objects :contentReference[oaicite:0]{index=0}

# 6) Ask Gemini to read all three before answering
user_prompt = """
Please read the attached papers and use them to fill in the JSON skeleton below. {kid_profile}
The JSON object should be a complete and accurate representation of the child's cognitive profile including the modalities, based on the provided scores and the information in the papers. Ensure that all fields are populated according to the guidelines in the system instruction prompt.
"""

response = model.generate_content(
    pdf_files + [user_prompt.format(kid_profile=profile1)], 
)

print(response.text)
completed_profile1 = response.text

{"name": "Kid", "gender": "male", "age": 13, "diagnosis": "Not formally diagnosed, but exhibiting early symptoms suggestive of attention deficit hyperactivity disorder (ADHD)", "cognitiveDomainProfile": {"workingMemory": {"score": 85, "description": "Kid's working memory is within the average range. He exhibits adequate skills in registering, maintaining, and manipulating both visual and auditory information in conscious awareness. He can effectively utilize these skills for problem-solving, demonstrating proficiency in recalling and sequencing series of pictures and lists of numbers."}, "processingSpeed": {"score": 78, "description": "Kid demonstrates average processing speed, indicating an adequate rate of mental operations. His reaction time to stimuli and completion time for mental activities are within the typical range for his age. This suggests efficient short-term memory scanning and inspection times for perceptual comparisons."}, "attentionSpan": {"visualAttention": {"score": 

In [6]:
response = model.generate_content(
    pdf_files + [user_prompt.format(kid_profile=profile2)], 
)

print(response.text)
completed_profile2 = response.text

{"name": "Kid", "gender": "female", "age": 13, "diagnosis": "Attention Deficit Hyperactivity Disorder (ADHD)", "cognitiveDomainProfile": {"workingMemory": {"score": 85, "description": "Kid's working memory is within the average range for her age, reflecting an age-appropriate ability to register, maintain, and manipulate visual and auditory information in conscious awareness.  She demonstrates adequate attention, concentration, and auditory and visual discrimination skills for her age."}, "processingSpeed": {"score": 78, "description": "Kid's processing speed is within the average range. This suggests an average rate at which she performs mental operations, including an average reaction time to stimuli and time needed to perform mental activities.  Her processing speed may be a relative weakness compared to other cognitive domains."}, "attentionSpan": {"visualAttention": {"score": 80, "description": "Kid's visual attention falls within the average range, suggesting adequate ability to 

In [7]:
response = model.generate_content(
    pdf_files + [user_prompt.format(kid_profile=profile3)], 
)

print(response.text)
completed_profile3 = response.text

{"name": "Kid", "gender": "female", "age": 13, "diagnosis": "Identified as Gifted and Talented (IQ > 130)", "cognitiveDomainProfile": {"workingMemory": {"score": 135, "description": "Kid exhibits an exceptional ability to register, maintain, and manipulate visual and auditory information in conscious awareness, indicative of superior attention, concentration, and mental control.  Her capacity for sequential processing, mental manipulation, and attention are also very strong. She appears to effectively utilize working memory in the service of learning, problem-solving, and higher-order reasoning."}, "processingSpeed": {"score": 140, "description": "Kid demonstrates remarkable speed and accuracy in visual identification, decision-making, and decision implementation.  Her rapid processing of visual information, quick and accurate decisions, and swift implementation of those decisions are evident.  This suggests that processing speed is not a limiting factor in her performance on tasks inv

In [8]:
response = model.generate_content(
    pdf_files + [user_prompt.format(kid_profile=profile4)], 
)

print(response.text)
completed_profile4 = response.text

{"name": "College Student", "gender": "female", "age": 21, "diagnosis": "Identified as Gifted and Talented (IQ > 130)", "cognitiveDomainProfile": {"workingMemory": {"score": 145, "description": "Exhibits exceptional working memory, retaining and manipulating multiple complex ideas simultaneously—great for advanced coursework and research."}, "processingSpeed": {"score": 150, "description": "Processes new information extremely quickly, routinely completing readings and problem sets faster than peers."}, "attentionSpan": {"visualAttention": {"score": 148, "description": "Maintains deep focus on visual materials (diagrams, slides, code) for extended periods without fatigue."}, "auditoryAttention": {"score": 149, "description": "Absorbs and integrates spoken lectures and discussions rapidly, rarely missing nuanced details even in fast‑paced seminars."}}}, "executiveFunction": {"planning": 10, "impulseControl": 10, "description": "Demonstrates immaculate organizational skills and self‑regul

In [9]:
pdf_files

[genai.File({
     'name': 'files/yhizpeksq7vn',
     'display_name': 'Assessing executive functions in children.pdf',
     'mime_type': 'application/pdf',
     'sha256_hash': 'MGRhZGQ3NTUwNGNjODdkNWIwZWVjY2ZlODllZjdjOTI3NjE3ZjFkZTYyZjJhMmZjODlmMjdkYzg4NDdmNWJlZg==',
     'size_bytes': '476328',
     'state': 'ACTIVE',
     'uri': 'https://generativelanguage.googleapis.com/v1beta/files/yhizpeksq7vn',
     'create_time': '2025-04-23T23:43:49.615879Z',
     'expiration_time': '2025-04-25T23:43:49.577465523Z',
     'update_time': '2025-04-23T23:43:49.615879Z'}),
 genai.File({
     'name': 'files/2ik3465njwo9',
     'display_name': 'Assessments Related to the Physical, Affective and Cognitive '
                     'Domains of Physical Literacy.pdf',
     'mime_type': 'application/pdf',
     'sha256_hash': 'ZDY3NzQyODdhMTZlMmY5MmMyZTljYTg0Njk2NzBjM2RmZDdkMjUwMWExNzI0MTQyODhmMTBmNDdjZWVhMmE3Zg==',
     'size_bytes': '1884494',
     'state': 'ACTIVE',
     'uri': 'https://generativelanguage.

## Tailoring Information

In [10]:
system_prompt_tailor_draft= """Task: Tailor the provided reference material so it’s engaging, accurate, and optimally suited to the learner’s cognitive profile.

Inputs:
- Reference Material:
  {{reference_materials}}

Instructions:
1. Profile Analysis
   Examine the learner_profile to extract:
   - Age and background (e.g. “college student,” “gifted child”)
   - Diagnosis or special considerations
   - Strengths & challenges in working memory, processing speed, attention (visual/auditory), executive function, etc.
   - Preferred learning modalities and motivational triggers

2. Format Selection
   Based on that profile, choose the one format that will work best:
   - Story or narrative
   - Dialogue or Q&A
   - Interactive activity or hands‑on exercise
   - Bulleted summary or “cheat‑sheet”
   - Real‑world analogy or case study
   - Visual/diagram description

   (Do not always default to a story—explain in one sentence why you picked your format.)

3. Content Adaptation
   - Break material into manageable “chunks” for limited working memory  
   - Use analogies, examples, or metaphors tied to the learner’s interests  
   - Insert prompts/questions or quick checks when helpful  
   - Simplify vocabulary and sentence structure as needed  

4. Tone & Engagement
   - Keep it encouraging and respectful  
   - Match the enthusiasm level of the learner (e.g. curious and playful for kids, concise and challenge‑driven for advanced students)  

5. Output
   - Provide only the tailored material in the chosen format  
   - Do not include cognative profile or any other information about the learner
   - Please do not include any additional commentary or instructions in the output. Just provide the tailored content.  
 
"""

In [22]:
system_prompt_tailor= """ Task: Tailor the provided reference material about photosynthesis to be easily understandable and engaging for a child with the following cognitive profile:

Reference Material:
{reference_materials}

Instructions:
1. Analyze the 'Child Cognitive Profile' to understand the child's age, diagnosis (if any), and specific strengths and weaknesses in working memory, processing speed, attention span (visual and auditory), executive functions (planning and impulse control), learning preferences (modalities), and the summary of learning challenges and strengths.
2. Based on this analysis, adapt the 'Reference Material' to be age-appropriate, engaging, and tailored to the child's cognitive needs and learning preferences.
3. Employ strategies to address any identified learning challenges, such as breaking down information into smaller chunks for limited working memory or incorporating visual aids for visual learners.
4. Maintain accuracy in the scientific information while simplifying the language and using analogies or relatable examples that resonate with a child.
5. Ensure the tone is encouraging, supportive, and avoids being condescending.
6. The output should be a clear and concise explanation of study materials suitable for the specific child described in the 'Child Cognitive Profile'.
7. Please do not include any additional commentary or instructions in the output. Just provide the tailored content.
8. Please do not include the cognitive profile in the output. Just provide the tailored content."""


user_prompt_tailor = """Here is the Kid Cognitive Profile:
{child_cognitive_profile_json}
Please tailor the provided reference material to be understandable and engaging for a child with the following cognitive profile.Good Luck!"""



In [12]:
with open("test_materials\photosynthesis.txt", "r", encoding="utf-8") as f:
    study_material = f.read()
print(study_material)

Photosynthesis is the process in which green plants (https://kids.britannica.com/kids/article/plant/353639) use sunlight to make their own food. Photosynthesis is necessary for life on Earth. Without it there would be no green plants, and without green plants there would be no animals.
Photosynthesis requires sunlight, chlorophyll, water, and carbon dioxide gas. Chlorophyll is a substance in all green plants, especially in the leaves. Plants take in water from the soil and carbon dioxide from the air.
Photosynthesis starts when chlorophyll absorbs energy (https://kids.britannica.com/kids/article/energy/353100) from sunlight. Green plants use this light energy to change water and carbon dioxide into oxygen and nutrients called sugars. The plants use some of the sugars and store the rest. The oxygen is released into the air.
Photosynthesis is very important because almost all living things depend on plants for food. Photosynthesis is also important because of the oxygen it produces. Huma

In [23]:
model = genai.GenerativeModel(
    model_name='models/gemini-1.5-flash',           
    system_instruction=system_prompt_tailor.format( reference_materials= study_material),
       
)

In [24]:

response1 = model.generate_content(
    user_prompt_tailor.format(child_cognitive_profile_json=completed_profile1),
)

display(Markdown(response1.text))

Plants are like tiny food factories!  They use a super cool process called photosynthesis to make their own food.  It's like magic, but it's science!

Imagine the sun is a giant lightbulb. Plants have a special green ingredient called chlorophyll – it's what makes leaves green – that acts like a solar panel, capturing the sun's energy.  Think of it like a plant's power source!

To make food, plants also need water (like you drink!) from the ground and carbon dioxide (a gas we breathe out) from the air.  The plant mixes these things with the sun's energy.

What do they make?  Sugar, which is the plant's food, and oxygen, which is the air we breathe!  It's like a recipe: sunlight + water + carbon dioxide = sugar + oxygen.

Plants use some of the sugar for energy to grow and store the rest for later. The oxygen gets released into the air, so we can all breathe!  Pretty amazing, right?

Lots of living things, not just plants, do photosynthesis. Some tiny water creatures and even some bacteria do it too! Photosynthesis is super important because without it, there wouldn't be enough oxygen for us or the animals!


In [25]:
response2 = model.generate_content(
    user_prompt_tailor.format(child_cognitive_profile_json=completed_profile2),
)

display(Markdown(response2.text))

Plants are like tiny food factories!  They make their own food using a super cool process called photosynthesis.  It's how they get energy to grow big and strong.

Imagine the sun as a giant lightbulb.  Plants have a special ingredient called chlorophyll, which is what makes leaves green. Chlorophyll acts like a solar panel, soaking up the sun's energy.

Plants also need water (like you drink!) from the ground and carbon dioxide (a gas we breathe out!) from the air.

The chlorophyll uses the sun's energy to mix the water and carbon dioxide together.  It's like a special recipe!  This makes sugary food for the plant and something super important for us – oxygen!  Plants let out the oxygen into the air, which we breathe in to stay alive.

So, next time you see a leafy green plant, remember it's a tiny food factory powered by the sun, making food for itself and giving us the air we need to breathe!  Pretty amazing, right?


In [30]:
response3 = model.generate_content(
    user_prompt_tailor.format(child_cognitive_profile_json=completed_profile3),
)

display(Markdown(response3.text))

Plants are like tiny food factories!  They use sunlight, water, and air to make their own food, a process called photosynthesis.  Think of it like this: sunlight is the energy, water is like the main ingredient, and air (specifically carbon dioxide) is a special ingredient.

Inside the leaves, there's a green stuff called chlorophyll.  Chlorophyll acts like a solar panel, capturing the sun's energy. This energy is used to mix the water and carbon dioxide into sugar (the plant's food!) and oxygen (the air we breathe!).

The plant uses some sugar for energy to grow big and strong, and it stores the rest for later.  The oxygen is released into the air – a super important byproduct, because we need oxygen to breathe!

Without photosynthesis, there wouldn't be plants, and without plants, there wouldn't be much food for animals, including us!  It's a pretty amazing process that keeps the whole world going.  Even some bacteria and algae use photosynthesis too!


In [28]:
response4 = model.generate_content(
    user_prompt_tailor.format(child_cognitive_profile_json=completed_profile4)
)

display(Markdown(response4.text))

Photosynthesis:  The Plant's Powerhouse

Plants are amazing! They create their own food using a process called photosynthesis. Think of it as their built-in kitchen and power plant all in one.  It's how they survive and provides food and oxygen for almost everything on Earth.

Here's how it works:  Plants need four key ingredients:

1. **Sunlight:**  Like a chef needs heat, plants need sunlight's energy.  They absorb this energy using a special green pigment called chlorophyll.  Chlorophyll is what makes leaves green!

2. **Water:**  Plants absorb water through their roots, like drinking a refreshing glass of water.

3. **Carbon Dioxide:**  Plants suck this gas from the air, just like we breathe in oxygen.

4. **Chlorophyll:** This is the amazing ingredient that traps the sunlight's energy. It's like a solar panel for the plant.


Using sunlight's energy, the chlorophyll acts like a tiny factory. It combines water and carbon dioxide to produce two things:

1. **Oxygen:** This is the air we breathe! Plants release oxygen into the atmosphere as a byproduct.

2. **Sugars (Glucose):** This is the plant's food!  It's like their delicious meal, providing energy for growth and everything else they need to do. They use some of the sugar immediately and store the rest for later.


Photosynthesis is crucial for life on Earth. Without it, there would be no plants, and without plants, there would be no animals (including us!). It's a vital process that keeps our planet alive and thriving.  Even some bacteria and algae use this same incredible process!
